# 回归拟合-大脑对齐（fmri->imgfeat）

In [1]:
import argparse, os
import numpy as np
from himalaya.backend import set_backend
from himalaya.ridge import RidgeCV
from himalaya.scoring import correlation_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
# 参数设置
target = 'dpt'   # init_latent / c
roi = ['early','ventral','midventral','midlateral','lateral','parietal']  # early  /  ventral
croi = 'midlateral'
backend = set_backend("numpy", on_error="warn")
subject='subj01'

In [3]:
# 路径设置
mridir = f'../data/fmri/{subject}'
featdir = f'../data/nsdfeat/{subject}_feat'

savedir = f'../data/decoded/{subject}'
os.makedirs(savedir, exist_ok=True)

In [4]:
# 超参数选择，正则化系数
alpha = [1e3, 3e3, 6e3, 9e3, 1e4, 3e4, 6e4, 9e4, 1e5]

In [5]:
'''算法流程'''
# 创建岭回归
ridge = RidgeCV(alphas=alpha)

# 创建预处理流程
preprocess_pipeline = make_pipeline(
    StandardScaler(with_mean=True, with_std=True),
)

# 预处理+岭回归 流程pipeline
pipeline = make_pipeline(
    preprocess_pipeline,
    ridge,
) 

In [6]:
'''加载训练和测试数据'''
X = []  # 训练样本（特有刺激响应），每个subj的所列roi的所有刺激响应体素值betas（9000*3=27000）
X_te = []  #测试样本（共有刺激大脑响应），每个subj的所列roi的平均刺激响应体素值betas（1000）
# for croi in roi:
#     # cX = np.load(f'{mridir}/{subject}_{croi}_betas_tr.npy').astype("float32")
#     cX = np.load(f'{mridir}/{subject}_{croi}_betas_ave_tr.npy').astype("float32")
#     cX_te = np.load(f'{mridir}/{subject}_{croi}_betas_ave_te.npy').astype("float32")
#     X.append(cX)
#     X_te.append(cX_te)
X.append(np.load(f'{mridir}/{subject}_{croi}_betas_ave_tr.npy').astype("float32"))
X_te.append(np.load(f'{mridir}/{subject}_{croi}_betas_ave_te.npy').astype("float32"))

# 堆叠所有roi并转为np
X = np.hstack(X)
X_te = np.hstack(X_te)

# 目标数据（图像特征）
# Y = np.load(f'{featdir}/{subject}_each_{target}_tr.npy').astype("float32").reshape([X.shape[0],-1])
Y = np.load(f'{featdir}/{subject}_ave_{target}_tr.npy').astype("float32").reshape([X.shape[0],-1])
Y_te = np.load(f'{featdir}/{subject}_ave_{target}_te.npy').astype("float32").reshape([X_te.shape[0],-1])

In [7]:
print(f'X {X.shape}, Y {Y.shape}, X_te {X_te.shape}, Y_te {Y_te.shape}')

X (9000, 834), Y (9000, 147456), X_te (1000, 834), Y_te (1000, 147456)


In [8]:
# 训练回归拟合模型
print(f'Now making decoding model for... {subject}:  {roi}, {target}')
pipeline.fit(X, Y)

Now making decoding model for... subj01:  ['early', 'ventral', 'midventral', 'midlateral', 'lateral', 'parietal'], dpt


Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('standardscaler', StandardScaler())])),
                ('ridgecv',
                 RidgeCV(alphas=[1000.0, 3000.0, 6000.0, 9000.0, 10000.0,
                                 30000.0, 60000.0, 90000.0, 100000.0]))])

In [9]:
# 模型预测和评估
brain_embs = pipeline.predict(X_te)
rs = correlation_score(Y_te.T,brain_embs.T)
print(f'Prediction accuracy is: {np.mean(rs):3.3}')

Prediction accuracy is: 0.276


In [10]:
# print(X_te[0])
# print(brain_embs[0])
# print(Y_te[0])
print(brain_embs.shape)

(1000, 147456)


In [11]:
best_alpha = pipeline.named_steps['ridgecv'].best_alphas_
print("Best alpha parameter:", best_alpha)

Best alpha parameter: [99999.97 99999.97 99999.97 ... 99999.97 99999.97 99999.97]


In [19]:
# 保存预测结果
np.save(f'{savedir}/{subject}_{"_".join(croi)}_brain_embs_{target}.npy',brain_embs)

### 下面是对齐后的解码

In [19]:
brain_align = np.load(f'../data/fmri_align/subj02/subj02_{roi[0]}_brain_align_{subject}.npy')

In [20]:
Y_te = np.load(f'../data/nsdfeat/subj02_feat/subj02_ave_{target}_te.npy').astype("float32")

In [21]:
brain_embs = pipeline.predict(brain_align)
rs = correlation_score(Y_te.T,brain_embs.T)
print(f'Prediction accuracy is: {np.mean(rs):3.3}')

Prediction accuracy is: 0.0481


In [23]:
np.save(f'../data/decoded/subj02/subj02_{"_".join(roi)}_brain_embs_{target}.npy',brain_embs)